In [1]:
import json
import os
import torch
from NetworkAnalysis.Create_heterogeneous_graph import Create_heterogeneous_graph
import pandas as pd

In [2]:
# Load configuration from the JSON file
with open('config/parameters.json', 'r') as config_file:
    config = json.load(config_file)

# Extract parameters with new structure
settings = config["settings"]
graph_params = config["graph_parameters"]
model_params = config["model_parameters"]
training_params = config["training_parameters"]

In [3]:
#read in (or create) heterodata_object
try:
    filepath = os.path.join(
        graph_params["base_path"],
        'multigraphs',
        f'heteroData_gene_cell_'
        f'{graph_params["cancer_type"].replace(" ", "_") if graph_params["cancer_type"] else "All"}_'
        f'{graph_params["gene_feat_name"]}_{graph_params["cell_feat_name"]}_'
        f'{"META" if graph_params["metapaths"] else ""}.pt'
    )
    heterodata_obj = torch.load(filepath)
    print(f"Loaded heterodata object from {filepath}")

except Exception as e:
    print(f"No file found, creating new one: {e}")
    graph_creator = Create_heterogeneous_graph(
        BASE_PATH=graph_params["base_path"],
        cancer_type=graph_params["cancer_type"],
        cell_feature=graph_params["cell_feat_name"],
        gene_feature=graph_params["gene_feat_name"],
        metapaths=graph_params["metapaths"]
    )
    heterodata_obj = graph_creator.run_pipeline()
    print(heterodata_obj)

No file found, creating new one: [Errno 2] No such file or directory: './Data/multigraphs/heteroData_gene_cell_Neuroblastoma_cgp_cnv_META.pt'
Loading data...
Continuing with Gene1 and Gene2 as columns for the nodes
14034 Nodes and 278974 interactions
Filtering data for network membership...
Filtering for informative genes...
Creating dependency network using informative genes...
Created dependency network with 25403 positive edges and 36958 negative edges
Ratio of negative to positive dependencies: 1.45
Processing gene features...
Processing cell features...
Processing features for 39 cells, feature type: cnv
Final network will have 13398 genes and 39 cell lines
Adding metapaths to the graph...
[[('gene', 'dependency_of', 'cell'), ('cell', 'rev_dependency_of', 'gene')], [('cell', 'rev_dependency_of', 'gene'), ('gene', 'dependency_of', 'cell')]]


/Users/jovanafilipovic/Downloads/MSc Bioinformatics/Year 2/Thesis/GIT_HetGNN/env/lib/python3.11/site-packages/torch_geometric/edge_index.py:2007: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:55.)
  out = torch.matmul(sparse_input, other)


HeteroData saved to ./Data/multigraphs/heteroData_gene_cell_Neuroblastoma_cgp_cnv_META.pt
HeteroData(
  metapath_dict={
    (gene, metapath_0, gene)=[2],
    (cell, metapath_1, cell)=[2],
  },
  gene={
    node_id=[13398],
    names=[13398],
    x=[13398, 3438],
  },
  cell={
    node_id=[39],
    names=[39],
    x=[39, 1224],
  },
  (gene, interacts_with, gene)={ edge_index=[2, 263122] },
  (gene, dependency_of, cell)={
    edge_index=[2, 61854],
    edge_label=[61854],
  },
  (gene, rev_interacts_with, gene)={ edge_index=[2, 263122] },
  (cell, rev_dependency_of, gene)={
    edge_index=[2, 61854],
    edge_label=[61854],
  },
  (gene, metapath_0, gene)={
    edge_index=[2, 2515396],
    edge_weight=[2515396],
  },
  (cell, metapath_1, cell)={
    edge_index=[2, 1521],
    edge_weight=[1521],
  }
)


In [14]:
# count dependecy edges with label 1 and 0 in heterodata_obj
lbls = heterodata_obj['gene', 'dependency_of', 'cell'].edge_label
print(len(lbls))

print(len(lbls[lbls == 1]))
print(len(lbls[lbls == 0]))

npr = len(lbls[lbls == 0])/len(lbls[lbls == 1])
print(npr)# load cell line metadata
ccles = pd.read_csv("Data/Depmap/Model.csv")



61854
25203
36651
1.4542316390905845


In [ ]:
# load cell line metadata
ccles = pd.read_csv("Data/Depmap/Model.csv")


In [5]:
print(ccles)

         ModelID  PatientID CellLineName StrippedCellLineName DepmapModelType  \
0     ACH-000001  PT-gj46wT  NIH:OVCAR-3            NIHOVCAR3           HGSOC   
1     ACH-000002  PT-5qa3uk        HL-60                 HL60             AML   
2     ACH-000003  PT-puKIyc        CACO2                CACO2            COAD   
3     ACH-000004  PT-q4K2cp          HEL                  HEL             AML   
4     ACH-000005  PT-q4K2cp   HEL 92.1.7              HEL9217             AML   
...          ...        ...          ...                  ...             ...   
1954  ACH-003161  PT-or1hkT    ABM-T9430             ABMT9430         ZIMMPSC   
1955  ACH-003181  PT-W75e4m     NRH-LMS1              NRHLMS1             LMS   
1956  ACH-003183  PT-BqidXH     NRH-MFS3              NRHMFS3             MFS   
1957  ACH-003184  PT-21NMVa     NRH-LMS2              NRHLMS2             LMS   
1958  ACH-003191  PT-B8KJKw     NRH-GCT2              NRHGCT2            GCTB   

           OncotreeLineage 